# Experimentos del Modelo de Ising
## Metropolis-Hastings vs Propp-Wilson Perfect Sampling

**Tarea 3 - Cadenas de Markov**

### Parámetros del Experimento:
- **Tamaños de lattice**: 10×10, 12×12, 15×15, 17×17, 20×20
- **β (temperatura inversa)**: 0, 0.1, 0.2, ..., 0.9, 1.0
- **J** = 1 (constante de acoplamiento)
- **B** = 0 (sin campo magnético externo)
- **100 muestras** de cada método
- **Metropolis-Hastings**: 10⁵ iteraciones por muestra

### Distribución de Boltzmann con Temperatura Inversa:
$$\pi(\sigma) = \frac{\exp(-\beta H(\sigma))}{Z(\beta)}$$

donde:
- $\beta = 1/T$ es la temperatura inversa
- $H(\sigma) = -J\sum_{\langle i,j \rangle} \sigma_i \sigma_j - B\sum_i \sigma_i$
- Con $J=1, B=0$: $H(\sigma) = -\sum_{\langle i,j \rangle} \sigma_i \sigma_j$

In [13]:
# Importar librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
import time
from typing import Dict, List
import warnings
warnings.filterwarnings('ignore')

# Configurar matplotlib
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Librerías importadas exitosamente")

Librerías importadas exitosamente


In [14]:
# Importar nuestros módulos
from ising_sampling import IsingModel, MetropolisHastings, ProppWilson, run_experiments, analyze_results
from test_ising import test_ising_model, test_metropolis_hastings, test_propp_wilson, test_boltzmann_distribution

print("Módulos del modelo de Ising importados exitosamente")

Módulos del modelo de Ising importados exitosamente


## 1. Pruebas Básicas del Modelo

In [15]:
# Ejecutar pruebas básicas
print("=== PRUEBAS BÁSICAS ===")
test_ising_model()
test_boltzmann_distribution()

=== PRUEBAS BÁSICAS ===
=== Prueba del Modelo de Ising ===
Lattice inicial 5x5:
[[ 1 -1  1  1  1]
 [ 1  1  1  1 -1]
 [-1 -1 -1 -1 -1]
 [ 1  1 -1  1  1]
 [ 1  1  1  1 -1]]
Magnetización: 7
Energía total: -6.0


=== Distribución de Boltzmann con β ===
π(σ) = exp(-β * H(σ)) / Z(β)
donde β = 1/T es la temperatura inversa
H(σ) = -J∑σᵢσⱼ - B∑σᵢ
Con J=1, B=0: H(σ) = -∑σᵢσⱼ (suma sobre vecinos)

Configuración de ejemplo 3x3:
[[ 1  1 -1]
 [ 1 -1 -1]
 [-1 -1  1]]
Energía H(σ) = 6.0

Probabilidades relativas para diferentes β:
β = 0.1: exp(-β*H) = 0.548812
β = 0.5: exp(-β*H) = 0.049787
β = 1.0: exp(-β*H) = 0.002479


In [16]:
# Pruebas de los algoritmos
test_metropolis_hastings()
test_propp_wilson()

=== Prueba Metropolis-Hastings ===
Configuración inicial:
[[ 1  1 -1  1  1]
 [-1  1  1 -1 -1]
 [ 1  1 -1 -1 -1]
 [ 1 -1  1  1 -1]
 [-1  1 -1  1  1]]
Energía inicial: 6.0

β = 0.1
Magnetización final: -1
Energía final: 6.0

β = 0.5
Magnetización final: -9
Energía final: -18.0

β = 1.0
Magnetización final: 25
Energía final: -50.0

=== Prueba Propp-Wilson ===

β = 0.2, lattice 4x4
Advertencia: No se alcanzó coalescencia en tiempo 100 para beta=0.2
Magnetización: 8
Energía: -12.0
Configuración:
[[ 1  1  1  1]
 [-1  1  1 -1]
 [-1 -1  1  1]
 [ 1  1  1  1]]

β = 0.8, lattice 4x4
Advertencia: No se alcanzó coalescencia en tiempo 100 para beta=0.8
Magnetización: -16
Energía: -32.0
Configuración:
[[-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [-1 -1 -1 -1]]


## 2. Experimento Completo

Ahora ejecutaremos el experimento completo con todos los parámetros especificados. **Nota**: Este experimento puede tomar varias horas.

In [17]:
# Función modificada para experimentos con progreso
def run_complete_experiments():
    """Ejecuta el experimento completo con todos los parámetros especificados"""
    
    # Parámetros completos del experimento
    lattice_sizes = [10, 12, 15, 17, 20]
    beta_values = [i * 0.1 for i in range(11)]  # 0, 0.1, 0.2, ..., 1.0
    n_samples = 100
    mh_steps = 100000  # 10^5 iteraciones
    
    print("=== EXPERIMENTO COMPLETO ===")
    print(f"Tamaños de lattice: {lattice_sizes}")
    print(f"Valores de β: {beta_values}")
    print(f"Número de muestras: {n_samples}")
    print(f"Pasos MH por muestra: {mh_steps}")
    print()
    
    total_experiments = len(lattice_sizes) * len(beta_values) * 2  # x2 for MH and PW
    current_experiment = 0
    
    results = {
        'metropolis_hastings': {},
        'propp_wilson': {},
        'parameters': {
            'lattice_sizes': lattice_sizes,
            'beta_values': beta_values,
            'n_samples': n_samples,
            'mh_steps': mh_steps,
            'J': 1.0,
            'B': 0.0
        }
    }
    
    start_total = time.time()
    
    for size in lattice_sizes:
        print(f"\n{'='*50}")
        print(f"LATTICE {size}x{size}")
        print(f"{'='*50}")
        
        results['metropolis_hastings'][size] = {}
        results['propp_wilson'][size] = {}
        
        for beta in beta_values:
            print(f"\nβ = {beta:.1f}")
            
            # METROPOLIS-HASTINGS
            print("  Ejecutando Metropolis-Hastings...")
            current_experiment += 1
            print(f"  Progreso: {current_experiment}/{total_experiments} ({100*current_experiment/total_experiments:.1f}%)")
            
            mh_samples = []
            mh_start = time.time()
            
            for sample_idx in range(n_samples):
                if (sample_idx + 1) % 25 == 0:
                    elapsed = time.time() - mh_start
                    estimated_total = elapsed * n_samples / (sample_idx + 1)
                    remaining = estimated_total - elapsed
                    print(f"    MH muestra {sample_idx + 1}/{n_samples} - Tiempo restante: {remaining:.1f}s")
                
                model = IsingModel(size, J=1.0, B=0.0)
                mh = MetropolisHastings(model)
                final_config = mh.run(beta, mh_steps, burn_in=10000)
                
                mh_samples.append({
                    'lattice': final_config.lattice.copy(),
                    'magnetization': final_config.magnetization(),
                    'energy': final_config.total_energy()
                })
            
            mh_time = time.time() - mh_start
            
            # PROPP-WILSON
            print("  Ejecutando Propp-Wilson...")
            current_experiment += 1
            print(f"  Progreso: {current_experiment}/{total_experiments} ({100*current_experiment/total_experiments:.1f}%)")
            
            pw_samples = []
            pw_start = time.time()
            
            for sample_idx in range(n_samples):
                if (sample_idx + 1) % 25 == 0:
                    elapsed = time.time() - pw_start
                    estimated_total = elapsed * n_samples / (sample_idx + 1)
                    remaining = estimated_total - elapsed
                    print(f"    PW muestra {sample_idx + 1}/{n_samples} - Tiempo restante: {remaining:.1f}s")
                
                pw = ProppWilson(size, J=1.0, B=0.0)
                final_config = pw.sample(beta, max_time=1000)
                
                pw_samples.append({
                    'lattice': final_config.lattice.copy(),
                    'magnetization': final_config.magnetization(),
                    'energy': final_config.total_energy()
                })
            
            pw_time = time.time() - pw_start
            
            # Almacenar resultados
            results['metropolis_hastings'][size][beta] = {
                'samples': mh_samples,
                'computation_time': mh_time
            }
            
            results['propp_wilson'][size][beta] = {
                'samples': pw_samples,
                'computation_time': pw_time
            }
            
            # Resumen rápido
            mh_mag = np.mean([s['magnetization'] for s in mh_samples])
            pw_mag = np.mean([s['magnetization'] for s in pw_samples])
            mh_energy = np.mean([s['energy'] for s in mh_samples])
            pw_energy = np.mean([s['energy'] for s in pw_samples])
            
            print(f"  Resultados: MH_mag={mh_mag:.2f}, PW_mag={pw_mag:.2f}")
            print(f"            MH_E={mh_energy:.2f}, PW_E={pw_energy:.2f}")
            print(f"  Tiempos: MH={mh_time:.1f}s, PW={pw_time:.1f}s")
            
            # Guardar progreso parcial
            if current_experiment % 10 == 0:
                with open('ising_results_partial.pkl', 'wb') as f:
                    pickle.dump(results, f)
                print(f"  >> Progreso guardado en ising_results_partial.pkl")
    
    total_time = time.time() - start_total
    
    # Guardar resultados finales
    with open('ising_results_complete.pkl', 'wb') as f:
        pickle.dump(results, f)
    
    print(f"\n{'='*50}")
    print("EXPERIMENTO COMPLETO FINALIZADO")
    print(f"Tiempo total: {total_time/3600:.2f} horas")
    print("Resultados guardados en 'ising_results_complete.pkl'")
    print(f"{'='*50}")
    
    return results

print("Función de experimento completo definida.")
print("Para ejecutar el experimento completo, ejecute la siguiente celda.")
print("ADVERTENCIA: El experimento puede tomar varias horas.")

Función de experimento completo definida.
Para ejecutar el experimento completo, ejecute la siguiente celda.
ADVERTENCIA: El experimento puede tomar varias horas.


In [18]:
# EJECUTAR EXPERIMENTO COMPLETO
# Descomente la siguiente línea para ejecutar el experimento completo
# ADVERTENCIA: Puede tomar varias horas

# complete_results = run_complete_experiments()

print("Para ejecutar el experimento completo, descomente la línea anterior.")
print("El experimento puede tomar varias horas dependiendo del hardware.")

Para ejecutar el experimento completo, descomente la línea anterior.
El experimento puede tomar varias horas dependiendo del hardware.


## 3. Análisis de Resultados

Una vez completado el experimento, podemos cargar y analizar los resultados.

In [19]:
def load_and_analyze_results(filename='ising_results_complete.pkl'):
    """Cargar y analizar resultados guardados"""
    try:
        with open(filename, 'rb') as f:
            results = pickle.load(f)
        print(f"Resultados cargados desde {filename}")
        return results
    except FileNotFoundError:
        print(f"Archivo {filename} no encontrado.")
        print("Ejecute primero el experimento completo.")
        return None

def comprehensive_analysis(results):
    """Análisis comprehensivo de los resultados"""
    if results is None:
        return
    
    sizes = results['parameters']['lattice_sizes']
    betas = results['parameters']['beta_values']
    n_samples = results['parameters']['n_samples']
    
    print("=== ANÁLISIS COMPREHENSIVO ===")
    print(f"Lattice sizes: {sizes}")
    print(f"Beta values: {betas}")
    print(f"Samples per configuration: {n_samples}")
    print()
    
    # Crear DataFrame para análisis
    data = []
    
    for size in sizes:
        for beta in betas:
            # Metropolis-Hastings
            mh_samples = results['metropolis_hastings'][size][beta]['samples']
            mh_time = results['metropolis_hastings'][size][beta]['computation_time']
            
            mh_mags = [s['magnetization'] for s in mh_samples]
            mh_energies = [s['energy'] for s in mh_samples]
            
            data.append({
                'size': size,
                'beta': beta,
                'method': 'Metropolis-Hastings',
                'magnetization_mean': np.mean(mh_mags),
                'magnetization_std': np.std(mh_mags),
                'abs_magnetization_mean': np.mean(np.abs(mh_mags)),
                'energy_mean': np.mean(mh_energies),
                'energy_std': np.std(mh_energies),
                'computation_time': mh_time
            })
            
            # Propp-Wilson
            pw_samples = results['propp_wilson'][size][beta]['samples']
            pw_time = results['propp_wilson'][size][beta]['computation_time']
            
            pw_mags = [s['magnetization'] for s in pw_samples]
            pw_energies = [s['energy'] for s in pw_samples]
            
            data.append({
                'size': size,
                'beta': beta,
                'method': 'Propp-Wilson',
                'magnetization_mean': np.mean(pw_mags),
                'magnetization_std': np.std(pw_mags),
                'abs_magnetization_mean': np.mean(np.abs(pw_mags)),
                'energy_mean': np.mean(pw_energies),
                'energy_std': np.std(pw_energies),
                'computation_time': pw_time
            })
    
    df = pd.DataFrame(data)
    
    # Mostrar resumen estadístico
    print("\nResumen estadístico por método:")
    print(df.groupby('method')[['magnetization_mean', 'energy_mean', 'computation_time']].describe())
    
    return df

# Intentar cargar resultados
results = load_and_analyze_results('ising_results_complete.pkl')

# Realizar análisis si hay resultados
if results is not None:
    df_analysis = comprehensive_analysis(results)
else:
    print("No hay resultados para analizar. Ejecute primero el experimento completo.")
    df_analysis = None

Archivo ising_results_complete.pkl no encontrado.
Ejecute primero el experimento completo.
No hay resultados para analizar. Ejecute primero el experimento completo.


## 4. Visualizaciones Avanzadas

In [20]:
def create_advanced_plots(results, df):
    """Crear visualizaciones avanzadas de los resultados"""
    
    if results is None or df is None:
        print("No hay datos para visualizar. Ejecute primero el experimento.")
        return
    
    fig = plt.figure(figsize=(20, 15))
    
    # Configuración de subplots
    gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)
    
    sizes = results['parameters']['lattice_sizes']
    betas = results['parameters']['beta_values']
    
    # 1. Magnetización absoluta vs Beta (por tamaño)
    ax1 = fig.add_subplot(gs[0, 0:2])
    
    for size in sizes:
        mh_data = df[(df['size'] == size) & (df['method'] == 'Metropolis-Hastings')]
        pw_data = df[(df['size'] == size) & (df['method'] == 'Propp-Wilson')]
        
        ax1.plot(mh_data['beta'], mh_data['abs_magnetization_mean'], 
                'o-', label=f'MH {size}x{size}', alpha=0.8)
        ax1.plot(pw_data['beta'], pw_data['abs_magnetization_mean'], 
                's--', label=f'PW {size}x{size}', alpha=0.8)
    
    ax1.set_xlabel('β (temperatura inversa)')
    ax1.set_ylabel('|Magnetización| promedio')
    ax1.set_title('Transición de Fase: Magnetización vs Temperatura')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Energía vs Beta
    ax2 = fig.add_subplot(gs[0, 2:4])
    
    for size in sizes:
        mh_data = df[(df['size'] == size) & (df['method'] == 'Metropolis-Hastings')]
        pw_data = df[(df['size'] == size) & (df['method'] == 'Propp-Wilson')]
        
        ax2.plot(mh_data['beta'], mh_data['energy_mean'], 
                'o-', label=f'MH {size}x{size}', alpha=0.8)
        ax2.plot(pw_data['beta'], pw_data['energy_mean'], 
                's--', label=f'PW {size}x{size}', alpha=0.8)
    
    ax2.set_xlabel('β (temperatura inversa)')
    ax2.set_ylabel('Energía promedio')
    ax2.set_title('Energía vs Temperatura')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Comparación de métodos (Magnetización)
    ax3 = fig.add_subplot(gs[1, 0:2])
    
    mh_data = df[df['method'] == 'Metropolis-Hastings']
    pw_data = df[df['method'] == 'Propp-Wilson']
    
    ax3.scatter(mh_data['abs_magnetization_mean'], pw_data['abs_magnetization_mean'], 
               c=mh_data['beta'], s=mh_data['size']*2, alpha=0.7, cmap='viridis')
    
    # Línea diagonal perfecta
    min_val = min(mh_data['abs_magnetization_mean'].min(), pw_data['abs_magnetization_mean'].min())
    max_val = max(mh_data['abs_magnetization_mean'].max(), pw_data['abs_magnetization_mean'].max())
    ax3.plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8, label='Perfecta concordancia')
    
    ax3.set_xlabel('Magnetización |M| - Metropolis-Hastings')
    ax3.set_ylabel('Magnetización |M| - Propp-Wilson')
    ax3.set_title('Comparación de Métodos: Magnetización')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Colorbar para el scatter
    cbar = plt.colorbar(ax3.collections[0], ax=ax3, shrink=0.8)
    cbar.set_label('β')
    
    # 4. Tiempos de cómputo
    ax4 = fig.add_subplot(gs[1, 2:4])
    
    # Box plot de tiempos por método
    method_times = []
    method_labels = []
    
    for method in ['Metropolis-Hastings', 'Propp-Wilson']:
        times = df[df['method'] == method]['computation_time'].values
        method_times.append(times)
        method_labels.append(method)
    
    ax4.boxplot(method_times, labels=method_labels)
    ax4.set_ylabel('Tiempo de cómputo (s)')
    ax4.set_title('Distribución de Tiempos de Cómputo')
    ax4.grid(True, alpha=0.3)
    
    # 5. Heatmap de magnetización (MH)
    ax5 = fig.add_subplot(gs[2, 0])
    
    mh_pivot = df[df['method'] == 'Metropolis-Hastings'].pivot(index='size', columns='beta', values='abs_magnetization_mean')
    im1 = ax5.imshow(mh_pivot.values, cmap='RdYlBu_r', aspect='auto')
    ax5.set_xticks(range(len(betas)))
    ax5.set_xticklabels([f'{b:.1f}' for b in betas])
    ax5.set_yticks(range(len(sizes)))
    ax5.set_yticklabels([f'{s}x{s}' for s in sizes])
    ax5.set_xlabel('β')
    ax5.set_ylabel('Tamaño')
    ax5.set_title('|M| - Metropolis-Hastings')
    plt.colorbar(im1, ax=ax5, shrink=0.8)
    
    # 6. Heatmap de magnetización (PW)
    ax6 = fig.add_subplot(gs[2, 1])
    
    pw_pivot = df[df['method'] == 'Propp-Wilson'].pivot(index='size', columns='beta', values='abs_magnetization_mean')
    im2 = ax6.imshow(pw_pivot.values, cmap='RdYlBu_r', aspect='auto')
    ax6.set_xticks(range(len(betas)))
    ax6.set_xticklabels([f'{b:.1f}' for b in betas])
    ax6.set_yticks(range(len(sizes)))
    ax6.set_yticklabels([f'{s}x{s}' for s in sizes])
    ax6.set_xlabel('β')
    ax6.set_ylabel('Tamaño')
    ax6.set_title('|M| - Propp-Wilson')
    plt.colorbar(im2, ax=ax6, shrink=0.8)
    
    # 7. Diferencia entre métodos
    ax7 = fig.add_subplot(gs[2, 2])
    
    diff_pivot = pw_pivot - mh_pivot
    im3 = ax7.imshow(diff_pivot.values, cmap='RdBu', aspect='auto', vmin=-0.1, vmax=0.1)
    ax7.set_xticks(range(len(betas)))
    ax7.set_xticklabels([f'{b:.1f}' for b in betas])
    ax7.set_yticks(range(len(sizes)))
    ax7.set_yticklabels([f'{s}x{s}' for s in sizes])
    ax7.set_xlabel('β')
    ax7.set_ylabel('Tamaño')
    ax7.set_title('Diferencia |M|\n(PW - MH)')
    plt.colorbar(im3, ax=ax7, shrink=0.8)
    
    # 8. Configuración de ejemplo
    ax8 = fig.add_subplot(gs[2, 3])
    
    # Mostrar configuración de ejemplo del caso más interesante
    example_size = sizes[len(sizes)//2] if len(sizes) > 1 else sizes[0]
    example_beta = betas[len(betas)//2] if len(betas) > 1 else betas[0]
    
    example_config = results['metropolis_hastings'][example_size][example_beta]['samples'][0]['lattice']
    im4 = ax8.imshow(example_config, cmap='RdBu', vmin=-1, vmax=1)
    ax8.set_title(f'Configuración Ejemplo\n{example_size}x{example_size}, β={example_beta}')
    ax8.set_xticks([])
    ax8.set_yticks([])
    plt.colorbar(im4, ax=ax8, shrink=0.8)
    
    plt.suptitle('Análisis Completo del Modelo de Ising', fontsize=16, y=0.98)
    plt.show()

# Crear visualizaciones avanzadas
if 'results' in globals() and 'df_analysis' in globals():
    create_advanced_plots(results, df_analysis)
else:
    print("No hay datos cargados para visualizar.")

No hay datos para visualizar. Ejecute primero el experimento.


## 5. Análisis Estadístico Detallado

In [21]:
def statistical_analysis(results, df):
    """Análisis estadístico detallado de los resultados"""
    
    if results is None or df is None:
        print("No hay datos para analizar. Ejecute primero el experimento.")
        return
    
    print("=== ANÁLISIS ESTADÍSTICO DETALLADO ===")
    print()
    
    # 1. Comparación de métodos
    print("1. COMPARACIÓN DE MÉTODOS")
    print("-" * 40)
    
    from scipy import stats
    
    # Test de diferencias en magnetización
    mh_mags = df[df['method'] == 'Metropolis-Hastings']['abs_magnetization_mean']
    pw_mags = df[df['method'] == 'Propp-Wilson']['abs_magnetization_mean']
    
    t_stat, p_value = stats.ttest_rel(mh_mags, pw_mags)
    
    print(f"Test t pareado para magnetización:")
    print(f"  t-statistic: {t_stat:.6f}")
    print(f"  p-value: {p_value:.6f}")
    
    if p_value < 0.05:
        print(f"  → Diferencia significativa entre métodos (p < 0.05)")
    else:
        print(f"  → No hay diferencia significativa entre métodos (p ≥ 0.05)")
    
    # Correlación entre métodos
    correlation, p_corr = stats.pearsonr(mh_mags, pw_mags)
    print(f"\nCorrelación entre métodos:")
    print(f"  Correlación de Pearson: {correlation:.6f}")
    print(f"  p-value: {p_corr:.6f}")
    
    # 2. Análisis por temperatura
    print("\n2. ANÁLISIS POR TEMPERATURA")
    print("-" * 40)
    
    betas = results['parameters']['beta_values']
    
    # Identificar temperatura crítica aproximada
    beta_c_approx = 2 / np.log(1 + np.sqrt(2))  # Valor teórico para red cuadrada
    print(f"Temperatura crítica teórica: β_c ≈ {beta_c_approx:.3f}")
    
    # Buscar el beta más cercano en nuestros datos
    closest_beta_idx = np.argmin(np.abs(np.array(betas) - beta_c_approx))
    closest_beta = betas[closest_beta_idx]
    print(f"Beta más cercano en experimento: {closest_beta:.1f}")
    
    # Análisis cerca de la transición
    transition_region = [b for b in betas if abs(b - beta_c_approx) < 0.3]
    print(f"Región de transición analizada: β ∈ {transition_region}")
    
    # 3. Análisis de eficiencia computacional
    print("\n3. EFICIENCIA COMPUTACIONAL")
    print("-" * 40)
    
    mh_times = df[df['method'] == 'Metropolis-Hastings']['computation_time']
    pw_times = df[df['method'] == 'Propp-Wilson']['computation_time']
    
    print(f"Metropolis-Hastings:")
    print(f"  Tiempo promedio: {mh_times.mean():.2f} ± {mh_times.std():.2f} s")
    print(f"  Rango: [{mh_times.min():.2f}, {mh_times.max():.2f}] s")
    
    print(f"\nPropp-Wilson:")
    print(f"  Tiempo promedio: {pw_times.mean():.2f} ± {pw_times.std():.2f} s")
    print(f"  Rango: [{pw_times.min():.2f}, {pw_times.max():.2f}] s")
    
    speedup = pw_times.mean() / mh_times.mean()
    if speedup > 1:
        print(f"\nMetropolis-Hastings es {speedup:.2f}x más rápido que Propp-Wilson")
    else:
        print(f"\nPropp-Wilson es {1/speedup:.2f}x más rápido que Metropolis-Hastings")
    
    # 4. Análisis por tamaño de sistema
    print("\n4. ESCALABILIDAD CON TAMAÑO DE SISTEMA")
    print("-" * 40)
    
    sizes = results['parameters']['lattice_sizes']
    
    for method in ['Metropolis-Hastings', 'Propp-Wilson']:
        print(f"\n{method}:")
        method_data = df[df['method'] == method]
        
        for size in sizes:
            size_data = method_data[method_data['size'] == size]
            avg_time = size_data['computation_time'].mean()
            avg_mag = size_data['abs_magnetization_mean'].mean()
            print(f"  {size}x{size}: tiempo={avg_time:.2f}s, |M|={avg_mag:.3f}")
    
    # 5. Resumen de validación
    print("\n5. VALIDACIÓN DE ALGORITMOS")
    print("-" * 40)
    
    # Calcular diferencias relativas
    rel_diff_mag = np.abs(mh_mags - pw_mags) / (np.abs(mh_mags) + 1e-10)
    
    print(f"Diferencia relativa en magnetización:")
    print(f"  Promedio: {rel_diff_mag.mean():.6f}")
    print(f"  Máxima: {rel_diff_mag.max():.6f}")
    print(f"  Percentil 95: {np.percentile(rel_diff_mag, 95):.6f}")
    
    if rel_diff_mag.mean() < 0.01:
        print(f"  ✓ Excelente concordancia entre métodos (<1% diferencia promedio)")
    elif rel_diff_mag.mean() < 0.05:
        print(f"  ✓ Buena concordancia entre métodos (<5% diferencia promedio)")
    else:
        print(f"  ⚠ Diferencias notables entre métodos (>5% diferencia promedio)")

# Ejecutar análisis estadístico
if 'results' in globals() and 'df_analysis' in globals():
    statistical_analysis(results, df_analysis)
else:
    print("No hay datos cargados para análisis estadístico.")

No hay datos para analizar. Ejecute primero el experimento.


## 6. Exportar Resultados

In [22]:
def export_results(results, df, filename_prefix='ising_results'):
    """Exportar resultados en diferentes formatos"""
    
    if results is None or df is None:
        print("No hay datos para exportar. Ejecute primero el experimento.")
        return
    
    print("=== EXPORTANDO RESULTADOS ===")
    
    # 1. DataFrame a CSV
    csv_filename = f'{filename_prefix}_summary.csv'
    df.to_csv(csv_filename, index=False)
    print(f"✓ Resumen exportado a {csv_filename}")
    
    # 2. Configuraciones de ejemplo
    configurations_data = []
    
    sizes = results['parameters']['lattice_sizes']
    betas = results['parameters']['beta_values']
    
    # Guardar algunas configuraciones de ejemplo
    for size in sizes[::2]:  # Cada segundo tamaño
        for beta in betas[::3]:  # Cada tercer beta
            for method in ['metropolis_hastings', 'propp_wilson']:
                sample = results[method][size][beta]['samples'][0]
                
                configurations_data.append({
                    'size': int(size),  # Convertir a int nativo de Python
                    'beta': float(beta),  # Convertir a float nativo de Python
                    'method': method,
                    'lattice': sample['lattice'].tolist(),
                    'magnetization': float(sample['magnetization']),  # Convertir a float nativo
                    'energy': float(sample['energy'])  # Convertir a float nativo
                })
    
    # Guardar configuraciones en JSON
    import json
    
    json_filename = f'{filename_prefix}_configurations.json'
    with open(json_filename, 'w') as f:
        json.dump(configurations_data, f, indent=2)
    print(f"✓ Configuraciones de ejemplo exportadas a {json_filename}")
    
    # 3. Resumen estadístico
    # Convertir todos los valores numpy a tipos nativos de Python para JSON
    def convert_numpy_types(obj):
        """Convierte tipos numpy a tipos nativos de Python"""
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {key: convert_numpy_types(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy_types(item) for item in obj]
        else:
            return obj
    
    summary_stats = {
        'experiment_parameters': convert_numpy_types(results['parameters']),
        'statistical_summary': {
            'by_method': convert_numpy_types(df.groupby('method')[['magnetization_mean', 'energy_mean', 'computation_time']].describe().to_dict()),
            'by_size': convert_numpy_types(df.groupby('size')[['magnetization_mean', 'energy_mean']].mean().to_dict()),
            'by_beta': convert_numpy_types(df.groupby('beta')[['magnetization_mean', 'energy_mean']].mean().to_dict())
        }
    }
    
    stats_filename = f'{filename_prefix}_statistics.json'
    with open(stats_filename, 'w') as f:
        json.dump(summary_stats, f, indent=2)
    print(f"✓ Estadísticas exportadas a {stats_filename}")
    
    # 4. Crear reporte en markdown
    report_filename = f'{filename_prefix}_report.md'
    
    with open(report_filename, 'w') as f:
        f.write("# Reporte de Experimentos del Modelo de Ising\\n\\n")
        f.write("## Parámetros del Experimento\\n\\n")
        f.write(f"- **Tamaños de lattice**: {sizes}\\n")
        f.write(f"- **Valores de β**: {betas}\\n")
        f.write(f"- **Número de muestras por configuración**: {results['parameters']['n_samples']}\\n")
        f.write(f"- **Iteraciones MH por muestra**: {results['parameters']['mh_steps']}\\n")
        f.write(f"- **J (acoplamiento)**: {results['parameters']['J']}\\n")
        f.write(f"- **B (campo magnético)**: {results['parameters']['B']}\\n\\n")
        
        f.write("## Distribución de Boltzmann\\n\\n")
        f.write("π(σ) = exp(-β H(σ)) / Z(β)\\n\\n")
        f.write("donde H(σ) = -J∑σᵢσⱼ - B∑σᵢ con J=1, B=0\\n\\n")
        
        f.write("## Resumen de Resultados\\n\\n")
        
        # Tabla de resumen por método
        method_summary = df.groupby('method')[['magnetization_mean', 'energy_mean', 'computation_time']].mean()
        
        f.write("### Promedios por Método\\n\\n")
        f.write("| Método | Magnetización | Energía | Tiempo (s) |\\n")
        f.write("|--------|---------------|---------|------------|\\n")
        
        for method, row in method_summary.iterrows():
            f.write(f"| {method} | {row['magnetization_mean']:.4f} | {row['energy_mean']:.2f} | {row['computation_time']:.2f} |\\n")
        
        f.write("\\n## Archivos Generados\\n\\n")
        f.write(f"- `{csv_filename}`: Datos tabulares completos\\n")
        f.write(f"- `{json_filename}`: Configuraciones de ejemplo\\n")
        f.write(f"- `{stats_filename}`: Estadísticas detalladas\\n")
        f.write(f"- `{report_filename}`: Este reporte\\n")
    
    print(f"✓ Reporte generado en {report_filename}")
    
    print("\\n=== EXPORTACIÓN COMPLETA ===")
    print(f"Archivos generados:")
    print(f"  - {csv_filename}")
    print(f"  - {json_filename}")
    print(f"  - {stats_filename}")
    print(f"  - {report_filename}")

# Exportar resultados
if 'results' in globals() and 'df_analysis' in globals():
    export_results(results, df_analysis)
else:
    print("No hay datos cargados para exportar.")

No hay datos para exportar. Ejecute primero el experimento.


## 7. Análisis Estadístico Detallado

In [23]:
def statistical_analysis(results, df):
    """Análisis estadístico detallado de los resultados"""
    
    print("=== ANÁLISIS ESTADÍSTICO DETALLADO ===")
    print()
    
    # 1. Comparación de métodos
    print("1. COMPARACIÓN DE MÉTODOS")
    print("-" * 40)
    
    from scipy import stats
    
    # Test de diferencias en magnetización
    mh_mags = df[df['method'] == 'Metropolis-Hastings']['abs_magnetization_mean']
    pw_mags = df[df['method'] == 'Propp-Wilson']['abs_magnetization_mean']
    
    t_stat, p_value = stats.ttest_rel(mh_mags, pw_mags)
    
    print(f"Test t pareado para magnetización:")
    print(f"  t-statistic: {t_stat:.6f}")
    print(f"  p-value: {p_value:.6f}")
    
    if p_value < 0.05:
        print(f"  → Diferencia significativa entre métodos (p < 0.05)")
    else:
        print(f"  → No hay diferencia significativa entre métodos (p ≥ 0.05)")
    
    # Correlación entre métodos
    correlation, p_corr = stats.pearsonr(mh_mags, pw_mags)
    print(f"\nCorrelación entre métodos:")
    print(f"  Correlación de Pearson: {correlation:.6f}")
    print(f"  p-value: {p_corr:.6f}")
    
    # 2. Análisis por temperatura
    print("\n2. ANÁLISIS POR TEMPERATURA")
    print("-" * 40)
    
    betas = results['parameters']['beta_values']
    
    # Identificar temperatura crítica aproximada
    beta_c_approx = 2 / np.log(1 + np.sqrt(2))  # Valor teórico para red cuadrada
    print(f"Temperatura crítica teórica: β_c ≈ {beta_c_approx:.3f}")
    
    # Buscar el beta más cercano en nuestros datos
    closest_beta_idx = np.argmin(np.abs(np.array(betas) - beta_c_approx))
    closest_beta = betas[closest_beta_idx]
    print(f"Beta más cercano en experimento: {closest_beta:.1f}")
    
    # Análisis cerca de la transición
    transition_region = [b for b in betas if abs(b - beta_c_approx) < 0.3]
    print(f"Región de transición analizada: β ∈ {transition_region}")
    
    # 3. Análisis de eficiencia computacional
    print("\n3. EFICIENCIA COMPUTACIONAL")
    print("-" * 40)
    
    mh_times = df[df['method'] == 'Metropolis-Hastings']['computation_time']
    pw_times = df[df['method'] == 'Propp-Wilson']['computation_time']
    
    print(f"Metropolis-Hastings:")
    print(f"  Tiempo promedio: {mh_times.mean():.2f} ± {mh_times.std():.2f} s")
    print(f"  Rango: [{mh_times.min():.2f}, {mh_times.max():.2f}] s")
    
    print(f"\nPropp-Wilson:")
    print(f"  Tiempo promedio: {pw_times.mean():.2f} ± {pw_times.std():.2f} s")
    print(f"  Rango: [{pw_times.min():.2f}, {pw_times.max():.2f}] s")
    
    speedup = pw_times.mean() / mh_times.mean()
    if speedup > 1:
        print(f"\nMetropolis-Hastings es {speedup:.2f}x más rápido que Propp-Wilson")
    else:
        print(f"\nPropp-Wilson es {1/speedup:.2f}x más rápido que Metropolis-Hastings")
    
    # 4. Análisis por tamaño de sistema
    print("\n4. ESCALABILIDAD CON TAMAÑO DE SISTEMA")
    print("-" * 40)
    
    sizes = results['parameters']['lattice_sizes']
    
    for method in ['Metropolis-Hastings', 'Propp-Wilson']:
        print(f"\n{method}:")
        method_data = df[df['method'] == method]
        
        for size in sizes:
            size_data = method_data[method_data['size'] == size]
            avg_time = size_data['computation_time'].mean()
            avg_mag = size_data['abs_magnetization_mean'].mean()
            print(f"  {size}x{size}: tiempo={avg_time:.2f}s, |M|={avg_mag:.3f}")
    
    # 5. Resumen de validación
    print("\n5. VALIDACIÓN DE ALGORITMOS")
    print("-" * 40)
    
    # Calcular diferencias relativas
    rel_diff_mag = np.abs(mh_mags - pw_mags) / (np.abs(mh_mags) + 1e-10)
    
    print(f"Diferencia relativa en magnetización:")
    print(f"  Promedio: {rel_diff_mag.mean():.6f}")
    print(f"  Máxima: {rel_diff_mag.max():.6f}")
    print(f"  Percentil 95: {np.percentile(rel_diff_mag, 95):.6f}")
    
    if rel_diff_mag.mean() < 0.01:
        print(f"  ✓ Excelente concordancia entre métodos (<1% diferencia promedio)")
    elif rel_diff_mag.mean() < 0.05:
        print(f"  ✓ Buena concordancia entre métodos (<5% diferencia promedio)")
    else:
        print(f"  ⚠ Diferencias notables entre métodos (>5% diferencia promedio)")

# Ejecutar análisis estadístico
if results is not None and df_analysis is not None:
    statistical_analysis(results, df_analysis)

## 8. Exportar Resultados

In [24]:
def export_results(results, df, filename_prefix='ising_results'):
    """Exportar resultados en diferentes formatos"""
    
    print("=== EXPORTANDO RESULTADOS ===")
    
    # 1. DataFrame a CSV
    csv_filename = f'{filename_prefix}_summary.csv'
    df.to_csv(csv_filename, index=False)
    print(f"✓ Resumen exportado a {csv_filename}")
    
    # 2. Configuraciones de ejemplo
    configurations_data = []
    
    sizes = results['parameters']['lattice_sizes']
    betas = results['parameters']['beta_values']
    
    # Guardar algunas configuraciones de ejemplo
    for size in sizes[::2]:  # Cada segundo tamaño
        for beta in betas[::3]:  # Cada tercer beta
            for method in ['metropolis_hastings', 'propp_wilson']:
                sample = results[method][size][beta]['samples'][0]
                
                configurations_data.append({
                    'size': int(size),  # Convertir a int nativo de Python
                    'beta': float(beta),  # Convertir a float nativo de Python
                    'method': method,
                    'lattice': sample['lattice'].tolist(),
                    'magnetization': float(sample['magnetization']),  # Convertir a float nativo
                    'energy': float(sample['energy'])  # Convertir a float nativo
                })
    
    # Guardar configuraciones en JSON
    import json
    
    json_filename = f'{filename_prefix}_configurations.json'
    with open(json_filename, 'w') as f:
        json.dump(configurations_data, f, indent=2)
    print(f"✓ Configuraciones de ejemplo exportadas a {json_filename}")
    
    # 3. Resumen estadístico
    # Convertir todos los valores numpy a tipos nativos de Python para JSON
    def convert_numpy_types(obj):
        """Convierte tipos numpy a tipos nativos de Python"""
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {key: convert_numpy_types(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy_types(item) for item in obj]
        else:
            return obj
    
    summary_stats = {
        'experiment_parameters': convert_numpy_types(results['parameters']),
        'statistical_summary': {
            'by_method': convert_numpy_types(df.groupby('method')[['magnetization_mean', 'energy_mean', 'computation_time']].describe().to_dict()),
            'by_size': convert_numpy_types(df.groupby('size')[['magnetization_mean', 'energy_mean']].mean().to_dict()),
            'by_beta': convert_numpy_types(df.groupby('beta')[['magnetization_mean', 'energy_mean']].mean().to_dict())
        }
    }
    
    stats_filename = f'{filename_prefix}_statistics.json'
    with open(stats_filename, 'w') as f:
        json.dump(summary_stats, f, indent=2)
    print(f"✓ Estadísticas exportadas a {stats_filename}")
    
    # 4. Crear reporte en markdown
    report_filename = f'{filename_prefix}_report.md'
    
    with open(report_filename, 'w') as f:
        f.write("# Reporte de Experimentos del Modelo de Ising\\n\\n")
        f.write("## Parámetros del Experimento\\n\\n")
        f.write(f"- **Tamaños de lattice**: {sizes}\\n")
        f.write(f"- **Valores de β**: {betas}\\n")
        f.write(f"- **Número de muestras por configuración**: {results['parameters']['n_samples']}\\n")
        f.write(f"- **Iteraciones MH por muestra**: {results['parameters']['mh_steps']}\\n")
        f.write(f"- **J (acoplamiento)**: {results['parameters']['J']}\\n")
        f.write(f"- **B (campo magnético)**: {results['parameters']['B']}\\n\\n")
        
        f.write("## Distribución de Boltzmann\\n\\n")
        f.write("π(σ) = exp(-β H(σ)) / Z(β)\\n\\n")
        f.write("donde H(σ) = -J∑σᵢσⱼ - B∑σᵢ con J=1, B=0\\n\\n")
        
        f.write("## Resumen de Resultados\\n\\n")
        
        # Tabla de resumen por método
        method_summary = df.groupby('method')[['magnetization_mean', 'energy_mean', 'computation_time']].mean()
        
        f.write("### Promedios por Método\\n\\n")
        f.write("| Método | Magnetización | Energía | Tiempo (s) |\\n")
        f.write("|--------|---------------|---------|------------|\\n")
        
        for method, row in method_summary.iterrows():
            f.write(f"| {method} | {row['magnetization_mean']:.4f} | {row['energy_mean']:.2f} | {row['computation_time']:.2f} |\\n")
        
        f.write("\\n## Archivos Generados\\n\\n")
        f.write(f"- `{csv_filename}`: Datos tabulares completos\\n")
        f.write(f"- `{json_filename}`: Configuraciones de ejemplo\\n")
        f.write(f"- `{stats_filename}`: Estadísticas detalladas\\n")
        f.write(f"- `{report_filename}`: Este reporte\\n")
    
    print(f"✓ Reporte generado en {report_filename}")
    
    print("\\n=== EXPORTACIÓN COMPLETA ===")
    print(f"Archivos generados:")
    print(f"  - {csv_filename}")
    print(f"  - {json_filename}")
    print(f"  - {stats_filename}")
    print(f"  - {report_filename}")

# Exportar resultados
if results is not None and df_analysis is not None:
    export_results(results, df_analysis)

## Conclusiones

Este notebook implementa y compara dos métodos de muestreo para el modelo de Ising:

1. **Metropolis-Hastings**: Algoritmo MCMC clásico con 10⁵ iteraciones por muestra
2. **Propp-Wilson**: Muestreo perfecto usando "coupling from the past"

### Distribución de Boltzmann con Temperatura Inversa

La distribución de equilibrio se expresa como:
$$\pi(\sigma) = \frac{\exp(-\beta H(\sigma))}{Z(\beta)}$$

donde $\beta = 1/T$ es la temperatura inversa y $H(\sigma) = -\sum_{\langle i,j \rangle} \sigma_i \sigma_j$ para $J=1, B=0$.

### Parámetros Experimentales

- **Lattice sizes**: 10×10, 12×12, 15×15, 17×17, 20×20
- **β values**: 0, 0.1, 0.2, ..., 0.9, 1.0
- **100 muestras** por configuración de cada método
- **Condiciones de frontera periódicas**

Los resultados permiten estudiar:
- Transiciones de fase ferromagnética
- Comparación de eficiencia entre métodos
- Validación de muestreo perfecto vs. aproximado
- Efectos de tamaño finito en el modelo de Ising